In [2]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np


In [15]:
# let's do a quick test to see how Random Forest is classifying these samples
df=pd.read_csv("../collapsed_datasets/honor_ds_3secs.csv")
df=df.drop("Unnamed: 0", axis=1)
X_test=df.drop("class", axis=1)
y_test=df["class"]
df_train=pd.read_csv("../collapsed_datasets/deviceMotion_3secs.csv")
X_train=df_train.drop("class", axis=1)
y_train=df_train["class"]




,rotationRate.x_mean,rotationRate.x_std,rotationRate.x_range,rotationRate.x_IRQ,rotationRate.y_mean,rotationRate.y_std,rotationRate.y_range,rotationRate.y_IRQ,rotationRate.z_mean,rotationRate.z_std,...,userAcceleration.zX#2,userAcceleration.zX#3,userAcceleration.zX#4,userAcceleration.zX#5,userAcceleration.zP#1,userAcceleration.zP#2,userAcceleration.zP#3,userAcceleration.zP#4,userAcceleration.zP#5,class
0,0.485156,2.762064,13.993376,3.305608,-0.243750,1.161159,5.050801,1.967597,-0.331189,1.034269,...,0.000533,0.000933,0.001333,0.001733,12.305169,26.259025,16.280097,9.847660,13.987488,downstairs
1,-0.377483,3.323853,14.792808,3.637126,0.166094,1.077427,4.642782,1.556743,0.209037,0.775714,...,0.000533,0.000933,0.001333,0.001600,25.222080,32.942613,39.680745,7.550469,12.473723,downstairs
2,0.058869,2.612729,12.318460,2.700546,0.073081,1.374832,7.546950,1.172717,0.030033,0.866680,...,0.000533,0.000933,0.001200,0.001600,20.811701,23.971477,26.198812,7.231452,14.303633,downstairs
3,-0.453245,3.096445,13.540454,3.291442,0.009436,1.561249,6.340389,2.244113,0.029219,0.950979,...,0.000533,0.000933,0.001333,0.001733,21.901189,33.924607,39.289062,9.603850,12.111498,downstairs
4,-0.283102,2.778656,12.016918,3.408461,0.040583,1.392356,6.053492,1.749705,0.033693,1.083285,...,0.000533,0.000933,0.001200,0.001600,22.444461,30.198514,28.820366,12.655176,10.843344,downstairs
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181,-0.112028,2.974215,10.208521,3.956529,-0.166129,1.504060,5.837639,2.272784,-0.221919,0.600366,...,0.000800,0.001067,0.001467,0.002000,25.972188,26.642463,6.197587,3.596096,7.630988,walking
182,-0.344927,2.734039,9.814355,2.259852,-1.580103,1.469321,6.287005,2.135665,-0.119584,0.479790,...,0.000800,0.001067,0.001333,0.001600,22.336937,19.942676,13.519746,7.736941,4.253725,walking
183,-0.209784,2.682671,9.603025,2.312401,-1.163761,1.393684,6.086950,1.669068,-0.023731,0.494402,...,0.000533,0.000800,0.001067,0.001600,7.120165,24.621680,15.600598,7.602581,6.307452,walking
184,0.588049,3.188333,10.232933,6.103522,0.275597,1.760986,8.056479,2.487079,-0.257274,0.802629,...,0.000533,0.000800,0.001067,0.001333,8.348899,26.248582,18.561503,10.859398,3.746034,walking
